In [1]:
import pandas as pd
import numpy as np
from  sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:

train = pd.read_csv('E:\\Dev\\AV\\train_votes.csv')
train = train.drop(train[train.Views > 3000000].index)


In [4]:
labelencoder_X = LabelEncoder()
train['Tag'] = labelencoder_X.fit_transform(train['Tag'])
train.drop(['ID','Username'], axis=1,inplace =True)
target = train['Upvotes']

In [5]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([train['Answers']])[0]
train['pd_watched'] = pd_watched

In [6]:
feature_names = [x for x in train.columns if x not in ['Upvotes']]
x_train, x_val, y_train, y_val = train_test_split(train[feature_names], target,test_size = 0.22,random_state =205)
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_val = sc_X.transform(x_val)

In [92]:
poly_reg = PolynomialFeatures(degree = 4,interaction_only=False, include_bias=True)
X_poly = poly_reg.fit_transform(x_train)
poly_reg.fit(x_train, y_train)
#lin_reg_1 = linear_model.LassoLars(alpha=0.0195,max_iter=350)#0.9030888558344501
#lin_reg_1 = linear_model.LassoLars(alpha=0.0490,max_iter=450)#0.9031607340990608
#lin_reg_1 = linear_model.LassoLars(alpha=0.0180,max_iter=160)#0.9027068526349653
#lin_reg_1 = linear_model.LassoLars(alpha=0.0208,max_iter=160)#0.903391895371656
#lin_reg_1 = linear_model.LassoLars(alpha=0.0205,max_iter=152)#0.9033236139262264
#lin_reg_1 = linear_model.LassoLars(alpha=0.02075,max_iter=152)#0.9033805906941818
lin_reg_1 = linear_model.LassoLars(alpha=0.02069,max_iter=160)#0.9033669851837212
#lin_reg_1 = linear_model.LassoLars(alpha=0.0198,max_iter=450)#0.9031607340990608
#lin_reg_1 = linear_model.LassoLars(alpha=0.0198,max_iter=450)#0.9031607340990608


lin_reg_1.fit(X_poly, y_train)

# predicitng 
pred_val = lin_reg_1.predict(poly_reg.fit_transform(x_val))

In [93]:
print(r2_score(y_val, pred_val))
# testing

test = pd.read_csv('E:\\Dev\\AV\\test_votes.csv')
ids = test['ID']
test.drop(['ID','Username'], axis=1,inplace =True)
#25-March - 0.903437879332785
#next 0.9034534322501852
#0.9037949838052866 - lb .794 

0.9033669851837212


In [77]:

labelencoder_X = LabelEncoder()
test['Tag'] = labelencoder_X.fit_transform(test['Tag'])

from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([test['Answers']])[0]
test['pd_watched'] = pd_watched

   

In [78]:

test = sc_X.fit_transform(test)

pred_test = lin_reg_1.predict(poly_reg.fit_transform(test))
pred_test=abs(pred_test)



In [79]:
import datetime

submission = pd.DataFrame({'ID': ids,
                           'Upvotes':pred_test
                           })

submission.to_csv("submit_" +str( datetime.date.today())+ "_nr.csv",index=False)

In [52]:
import os
cwd = os.getcwd()
cwd

'C:\\Users\\Narayan'

In [26]:
0.9034534322501852 - 0.903437879332785 

1.5552917400185606e-05

In [38]:
def train_lassolars_model(train_x, train_y, predict_x):
    print_title("LassoLars Regressor")
    reg = linear_model.LassoLarsCV(
        cv=10, n_jobs=3, max_iter=2000, normalize=False)
    reg.fit(train_x, train_y)
    print("alphas and cv_alphas: {0} and {1}".format(
        reg.alphas_.shape, reg.cv_alphas_.shape))
    print("alphas[%d]: %s" % (len(reg.cv_alphas_), reg.cv_alphas_))
    print("mse shape: {0}".format(reg.cv_mse_path_.shape))
    # print("mse: %s" % np.mean(_mse, axis=0))
    # print("mse: %s" % np.mean(_mse, axis=1))
    # index = np.where(reg.alphas_ == reg.alpha_)
    # print("itemindex: %s" % index)
    index = np.where(reg.cv_alphas_ == reg.alpha_)
    _mse_v = np.mean(reg.cv_mse_path_[index, :])
    print("mse value: %f" % _mse_v)

    print("best alpha: %f" % reg.alpha_)
    best_alpha = reg.alpha_
    reg = linear_model.LassoLars(alpha=best_alpha)
    reg.fit(train_x, train_y)
    n_nonzeros = (reg.coef_ != 0).sum()
    print("Non-zeros coef: %d" % n_nonzeros)
    predict_y = reg.predict(predict_x)
    return {'y': predict_y, "coef": reg.coef_} 